In [1]:
import pandas as pd
import numpy as np

In [23]:
df_train = pd.read_csv('data/train.csv')

In [24]:
label = df_train.label.values.copy()
pixels = df_train.drop('label', axis=1).values.copy() / 255.
pixels = pixels.reshape(len(pixels), 28, 28)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pixels, label, test_size=0.15, random_state=101)
len(X_train), len(X_test)

(35700, 6300)

## Model

In [41]:
from keras.layers import Softmax, ReLU, Dense, Flatten, Dropout, MaxPool2D, Bidirectional
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K
K.set_image_dim_ordering('th')

In [77]:
model = Sequential()

model.add(Bidirectional(LSTM(100, input_shape=(28, 28), recurrent_dropout=0.25, return_sequences=True)))
model.add(Bidirectional(LSTM(100, input_shape=(28, 28), recurrent_dropout=0.25)))

model.add(Dense(128))
model.add(Dropout(0.25))


model.add(Dense(10))
model.add(Softmax())

In [78]:
model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

ValueError: This model has not yet been built. Build the model first by calling build() or calling fit() with some data. Or specify input_shape or batch_input_shape in the first layer for automatic build. 

In [79]:
Y_train = to_categorical(y_train, num_classes=10)
Y_test = to_categorical(y_test, num_classes=10)

In [ ]:
model.fit(X_train, Y_train, batch_size=200,
            validation_data = (X_test, Y_test),
            epochs=10)
# , steps_per_epoch=len(X_train)//200, validation_steps=len(X_test)//200)

Exception ignored in: <bound method TF_Output.<lambda> of <tensorflow.python.pywrap_tensorflow_internal.TF_Output; proxy of <Swig Object of type 'TF_Output *' at 0x0000019D6695C1B0> >>
Traceback (most recent call last):
  File "C:\Users\joavi\Anaconda3\envs\py36\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 1058, in <lambda>
    __del__ = lambda self: None
KeyboardInterrupt: 


Train on 35700 samples, validate on 6300 samples
Epoch 1/10
35700/35700 [==============================] - 50s 1ms/step - loss: 1.8369 - acc: 0.3797 - val_loss: 1.2122 - val_acc: 0.5978
Epoch 2/10
27200/35700 [=====================>........] - ETA: 9s - loss: 1.1215 - acc: 0.6224

In [56]:
test_df = pd.read_csv('data/test.csv')

In [257]:
test_df.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [258]:
test_values = test_df.values.copy()

In [259]:
test_values = test_values.reshape(len(test_values), 1, 28, 28)

In [260]:
from tqdm import tqdm

In [261]:
preds = []
for i in tqdm(range(0, len(test_values), 10)):
    pr = model.predict(test_values[i:i+10])
#     print(pr)
    preds.extend(list(np.argmax(pr, axis=1)))

100%|██████████| 2800/2800 [00:15<00:00, 185.83it/s]


In [262]:
df_preds = pd.DataFrame(list(zip([i+1 for i in range(len(preds))], preds)), columns=['ImageId', 'Label'])

In [263]:
df_preds.to_csv('predictions.csv', index=False)